<a href="https://colab.research.google.com/github/ahamednazeer/AI/blob/master/6G_technologies_for_healthcare.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# Step 1: Set up the environment in Colab
!pip install kaggle tensorflow numpy flask pyngrok

# Step 2: Check if Google Drive is mounted
from google.colab import drive
try:
    drive.mount('/content/drive')
except:
    print("Drive already mounted or error occurred. Proceeding...")

# Step 3: Set up Kaggle API
from google.colab import files
print("Please upload kaggle.json")
uploaded = files.upload()
kaggle_file = next(iter(uploaded))
print(f"Uploaded file: {kaggle_file}")
!mkdir -p ~/.kaggle
!cp "{kaggle_file}" ~/.kaggle/kaggle.json
!chmod 600 ~/.kaggle/kaggle.json

# Step 4: Download the Chest X-ray Pneumonia dataset
!kaggle datasets download -d paultimothymooney/chest-xray-pneumonia
!unzip -q chest-xray-pneumonia.zip -d /content/chest_xray

# Step 5: Import libraries
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import time
import random
import os
import shutil
from glob import glob
from flask import Flask, render_template_string
from pyngrok import ngrok
import threading

# Step 6: Create a small dataset subset
def create_small_dataset(source_dir, target_dir, num_train=500, num_test=100):
    print(f"Source directory: {source_dir}")
    os.makedirs(target_dir, exist_ok=True)
    os.makedirs(os.path.join(target_dir, 'train/NORMAL'), exist_ok=True)
    os.makedirs(os.path.join(target_dir, 'train/PNEUMONIA'), exist_ok=True)
    os.makedirs(os.path.join(target_dir, 'test/NORMAL'), exist_ok=True)
    os.makedirs(os.path.join(target_dir, 'test/PNEUMONIA'), exist_ok=True)

    normal_train = glob(os.path.join(source_dir, 'train/NORMAL/*.jpeg'))
    pneumonia_train = glob(os.path.join(source_dir, 'train/PNEUMONIA/*.jpeg'))
    normal_test = glob(os.path.join(source_dir, 'test/NORMAL/*.jpeg'))
    pneumonia_test = glob(os.path.join(source_dir, 'test/PNEUMONIA/*.jpeg'))

    print(f"Found {len(normal_train)} NORMAL training images")
    print(f"Found {len(pneumonia_train)} PNEUMONIA training images")
    print(f"Found {len(normal_test)} NORMAL test images")
    print(f"Found {len(pneumonia_test)} PNEUMONIA test images")

    normal_train = normal_train[:min(num_train//2, len(normal_train))]
    pneumonia_train = pneumonia_train[:min(num_train//2, len(pneumonia_train))]
    normal_test = normal_test[:min(num_test//2, len(normal_test))]
    pneumonia_test = pneumonia_test[:min(num_test//2, len(pneumonia_test))]

    for img in normal_train:
        shutil.copy(img, os.path.join(target_dir, 'train/NORMAL'))
    for img in pneumonia_train:
        shutil.copy(img, os.path.join(target_dir, 'train/PNEUMONIA'))
    for img in normal_test:
        shutil.copy(img, os.path.join(target_dir, 'test/NORMAL'))
    for img in pneumonia_test:
        shutil.copy(img, os.path.join(target_dir, 'test/PNEUMONIA'))

    print(f"Copied {len(glob(os.path.join(target_dir, 'train/NORMAL/*.jpeg')))} NORMAL training images")
    print(f"Copied {len(glob(os.path.join(target_dir, 'train/PNEUMONIA/*.jpeg')))} PNEUMONIA training images")
    print(f"Copied {len(glob(os.path.join(target_dir, 'test/NORMAL/*.jpeg')))} NORMAL test images")
    print(f"Copied {len(glob(os.path.join(target_dir, 'test/PNEUMONIA/*.jpeg')))} PNEUMONIA test images")

# Step 7: Simulate 5G and 6G network characteristics
class Network:
    def __init__(self, network_type="6G"):
        if network_type == "6G":
            self.latency_ms = random.uniform(0.1, 1.0)  # 6G: 0.1–1 ms
            self.bandwidth_gbps = 1.0  # 6G: 1 Gbps
            self.reliability = 0.99999  # 6G: 99.999%
        else:  # 5G
            self.latency_ms = random.uniform(1.0, 10.0)  # 5G: 1–10 ms
            self.bandwidth_gbps = 0.1  # 5G: 100 Mbps
            self.reliability = 0.999  # 5G: 99.9%
        self.network_type = network_type

    def transmit_data(self, data_size_mb):
        transmission_time = (data_size_mb * 8) / self.bandwidth_gbps / 1000
        transmission_time += self.latency_ms / 1000
        if random.random() > self.reliability:
            raise Exception(f"Transmission failed due to {self.network_type} network error!")
        return transmission_time

# Step 8: Load the small dataset
def load_small_dataset(data_dir, img_size=(224, 224), batch_size=16):
    datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=10,
        width_shift_range=0.1,
        height_shift_range=0.1,
        zoom_range=0.1
    )

    train_generator = datagen.flow_from_directory(
        os.path.join(data_dir, 'train'),
        target_size=img_size,
        batch_size=batch_size,
        class_mode='binary',
        classes=['NORMAL', 'PNEUMONIA'],
        shuffle=True
    )

    test_generator = datagen.flow_from_directory(
        os.path.join(data_dir, 'test'),
        target_size=img_size,
        batch_size=batch_size,
        class_mode='binary',
        classes=['NORMAL', 'PNEUMONIA'],
        shuffle=False
    )

    if train_generator.samples == 0 or test_generator.samples == 0:
        raise ValueError(f"No images found in the dataset. Check directory: {data_dir}")

    return train_generator, test_generator

# Step 9: Build transfer learning model
def build_transfer_learning_model():
    base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    base_model.trainable = False
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(64, activation='relu')(x)
    x = tf.keras.layers.Dropout(0.5)(x)
    predictions = Dense(1, activation='sigmoid')(x)
    model = Model(inputs=base_model.input, outputs=predictions)
    model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Step 10: Simulate federated learning with 5G/6G
def federated_learning_simulation(model, train_generator, num_devices=3, num_rounds=3, epochs_per_round=1, network_type="6G"):
    global_weights = model.get_weights()
    batch_size = train_generator.batch_size
    total_samples = train_generator.samples
    samples_per_device = total_samples // num_devices
    transmission_times = []

    for round in range(num_rounds):
        print(f"\nFederated Learning Round {round + 1}/{num_rounds} ({network_type})")
        local_models = []

        for device_id in range(num_devices):
            print(f"Training on IoMT Device {device_id + 1}")
            model.set_weights(global_weights)
            network = Network(network_type)
            data_size_mb = (samples_per_device * 224 * 224 * 3 * 32) / (8 * 1024 * 1024)
            transmission_time = network.transmit_data(data_size_mb)
            transmission_times.append(transmission_time)
            print(f"Data transmission time ({network_type}): {transmission_time:.4f} seconds")

            start_time = time.time()
            steps_per_epoch = max(1, samples_per_device // batch_size)
            model.fit(train_generator, epochs=epochs_per_round, steps_per_epoch=steps_per_epoch, verbose=0)
            print(f"Local training time on Device {device_id + 1}: {time.time() - start_time:.4f} seconds")
            local_models.append(model.get_weights())

        global_weights = [np.mean([local_weights[i] for local_weights in local_models], axis=0)
                         for i in range(len(global_weights))]
        model.set_weights(global_weights)

    return model, transmission_times

# Step 11: Flask app for dashboard
app = Flask(__name__)
results = {}

@app.route('/')
def dashboard():
    html = """
    <!DOCTYPE html>
    <html>
    <head>
        <title>Pneumonia Detection: 6G vs 5G Simulation</title>
        <style>
            body { font-family: Arial, sans-serif; margin: 20px; background-color: #f0f0f0; }
            h1 { color: #333; }
            table { border-collapse: collapse; width: 80%; margin: 20px auto; }
            th, td { border: 1px solid #ccc; padding: 10px; text-align: center; }
            th { background-color: #4CAF50; color: white; }
            tr:nth-child(even) { background-color: #f2f2f2; }
        </style>
    </head>
    <body>
        <h1>Pneumonia Detection: 6G vs 5G Simulation</h1>
        <table>
            <tr>
                <th>Network</th>
                <th>Test Accuracy (%)</th>
                <th>Test Loss</th>
                <th>Avg Transmission Time (s)</th>
            </tr>
            {% for network, data in results.items() %}
            <tr>
                <td>{{ network }}</td>
                <td>{{ data.accuracy }}</td>
                <td>{{ data.loss }}</td>
                <td>{{ data.avg_transmission_time }}</td>
            </tr>
            {% endfor %}
        </table>
    </body>
    </html>
    """
    return render_template_string(html, results=results)

# Step 12: Main simulation
def main():
    global results

    # Create small dataset
    print("Creating small dataset...")
    source_dir = '/content/chest_xray/chest_xray'
    target_dir = '/content/small_chest_xray'
    create_small_dataset(source_dir, target_dir, num_train=500, num_test=100)

    # Load the small dataset
    print("Loading small Pneumonia dataset...")
    try:
        train_generator, test_generator = load_small_dataset(target_dir, batch_size=16)
    except ValueError as e:
        print(f"Error: {e}")
        return

    # Run simulations for 6G and 5G
    for network_type in ["6G", "5G"]:
        print(f"\nSimulating {network_type} network...")
        model = build_transfer_learning_model()
        start_time = time.time()
        model, transmission_times = federated_learning_simulation(
            model, train_generator, num_devices=3, num_rounds=3, network_type=network_type
        )
        total_time = time.time() - start_time

        # Evaluate the model
        loss, accuracy = model.evaluate(test_generator, verbose=0)
        avg_transmission_time = np.mean(transmission_times)

        results[network_type] = {
            'accuracy': f"{accuracy * 100:.2f}",
            'loss': f"{loss:.4f}",
            'avg_transmission_time': f"{avg_transmission_time:.4f}"
        }
        print(f"{network_type} - Test Accuracy: {accuracy * 100:.2f}%, Test Loss: {loss:.4f}, "
              f"Avg Transmission Time: {avg_transmission_time:.4f} seconds, Total Time: {total_time:.4f} seconds")

    # Start Flask app in a separate thread
    print("\nStarting Flask app...")
    threading.Thread(target=lambda: app.run(host='0.0.0.0', port=5000)).start()

    # Set up Ngrok
    print("Please enter your Ngrok authtoken (from https://dashboard.ngrok.com/get-started/your-authtoken):")
    ngrok_token = input()
    ngrok.set_auth_token(ngrok_token)
    public_url = ngrok.connect(5000).public_url
    print(f"Dashboard is publicly accessible at: {public_url}")

if __name__ == "__main__":
    main()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Please upload kaggle.json


Saving kaggle.json to kaggle (2).json
Uploaded file: kaggle (2).json
Dataset URL: https://www.kaggle.com/datasets/paultimothymooney/chest-xray-pneumonia
License(s): other
chest-xray-pneumonia.zip: Skipping, found more recently modified local copy (use --force to force download)
replace /content/chest_xray/chest_xray/__MACOSX/._chest_xray? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
Creating small dataset...
Source directory: /content/chest_xray/chest_xray
Found 1341 NORMAL training images
Found 3875 PNEUMONIA training images
Found 234 NORMAL test images
Found 390 PNEUMONIA test images
Copied 250 NORMAL training images
Copied 250 PNEUMONIA training images
Copied 50 NORMAL test images
Copied 50 PNEUMONIA test images
Loading small Pneumonia dataset...
Found 500 images belonging to 2 classes.
Found 100 images belonging to 2 classes.

Simulating 6G network...

Federated Learning Round 1/3 (6G)
Training on IoMT Device 1
Data transmission time (6G): 0.7635 seconds
Local training time on Device 1:

Address already in use
Port 5000 is in use by another program. Either identify and stop that program, or start the server with a different port.


2YsmHMOy8PrZC6e17Q0XxAVdvTt_6iSPDiqyJfh2Uu4hZtvqb
Dashboard is publicly accessible at: https://e1f08b4b0482.ngrok-free.app
